< [How to Read and Represent Data](../ica02/How_to_Read_and_Represent_Data.ipynb) | Contents (TODO) | Next Material (TODO) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica03/Data_Mining.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# Data Mining

Data mining is ... (TODO)

# 1. Kaggle Datasets

## 1.1. What is Kaggle?

## 1.2. Getting Started

### Sign up


### Create an API Token

![User settings menu](figures/kaggle_my_account.png)

Scroll down a bit and you will find an API section. Click '`Create New API Token`' button.

![Create New Kaggle API Token](figures/create_new_kaggle_api_token.png)


### Copy JSON file

In [1]:
# This is how you install a python library
!pip install kaggle

In [ ]:
# type your Kaggle API token here
token = {"username":"bigdata","key":"a1b2c3d4"}

In [ ]:
# This cell is only for advanced users. Run this cell and you can safely move on to the next cell
import os
from pathlib import Path
import json
import platform

# creates and places the token file at a desired location
home = str(Path.home())
kaggle_root = os.path.join(*[home, '.kaggle'])
os.mkdir(kaggle_root)
with open(os.path.join(*[kaggle_root, 'kaggle.json']), 'w') as file:
    json.dump(token, file)

# make the key file accessible only to the owner
if platform.system() == 'Windows':
    !attrib -R {os.path.join(*[kaggle_root, 'kaggle.json'])}
else:
    !chmod 600 {os.path.join(*[kaggle_root, 'kaggle.json'])}

In [ ]:
# This is how you download datasets from Kaggle
# !kaggle datasets download -d <path-to-dataset> -p <download-location>
#    <path-to-dataset> is what comes after http://www.kaggle.com/ in the dataset URL.
#                      for example, the avocado dataset sits under https://www.kaggle.com/neuromusic/avocado-prices
#    <download-location> is where you want to download data.
!kaggle datasets download -d neuromusic/avocado-prices -p data

In [ ]:
# Unzipping files.
if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])
    print('[IMPORTANT] No automatic unzipping supported on Windows.')
    print('You have to open `File Explorer` and manually unzip `' + data_path + '\\avocado-prices.zip`')
    print('Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:')
    print('|- ica02')
    print('    |- data')
    print('        |- avocado.csv')
    print('    |- How_to_Read_and_Represent_Data.ipynb')
else:
    !unzip ./data/new-york-city-current-job-postings.zip ./data

In [ ]:
import pandas as pd

DF = pd.read_csv('data/avocado.csv')
DF.head(5)

In [ ]:
DF['Total Bags']

## Web Crawling with LXML
`lxml` is a package ... (TODO)

It parses the hyperlink structure of a webpage so that you can navigate through the links. `lxml` comes with the default Anaconda Python distribution, but it can also be installed by running:
```
!pip install lxml
```
if it is not already available in your system.

There are so many ways `lxml` can be used for web crawling, but in this notebook, I show a very simple example of using `html` parser module in `lxml` as an introduction. The `html` module under `lxml` package can be imported as:

In [5]:
import lxml.html

`html` module provides a function `parse()` which parses html tags. Parsing html tags in a webpage is as simple as:

In [27]:
url = 'http://user.engineering.uiowa.edu/~sbaek/about.html'
tree = lxml.html.parse(url)
print(lxml.html.tostring(tree))

b'<!DOCTYPE html>\n<html lang="en">\r\n<head>\r\n\r\n  <meta charset="utf-8">\r\n  <title>About | Stephen Baek</title>\r\n  <!--<link rel="icon" type="image/png" href="img/icon.png" />-->\r\n\r\n<!--Style-->\r\n\r\n  <link rel="stylesheet" href="css/reset.css">\r\n  <link rel="stylesheet" href="css/style.css">\r\n  <link rel="stylesheet" href="css/style-responsive.css">\r\n  <link rel="stylesheet" href="http://maxcdn.bootstrapcdn.com/font-awesome/4.3.0/css/font-awesome.min.css">\r\n\r\n  <!--[if lt IE 9]>\r\n  <script src="http://html5shiv.googlecode.com/svn/trunk/html5.js"></script>\r\n  <![endif]-->\r\n\r\n</head>\r\n\r\n<body>\r\n\r\n<!--Preloader-->\r\n\r\n<!--\r\n<div class="preloader" id="preloader">\r\n    <div class="item">\r\n      <div class="spinner">\r\n      </div>\r\n    </div>\r\n</div>\r\n-->\r\n<div class="opacity-nav" id="nav"></div>\r\n<header id="header"><!-- id="fullscreen">--></header>\r\n\r\n  <div class="clear"></div>\r\n\r\n  <!--Content-->\r\n\r\n  <div class=

In [28]:
root = tree.getroot()
links = root.findall('.//a')
print(links)

[<Element a at 0x255fac21f48>, <Element a at 0x255faccdd18>, <Element a at 0x255faccda98>, <Element a at 0x255faccdae8>, <Element a at 0x255faccdbd8>, <Element a at 0x255faccdc28>, <Element a at 0x255faccdc78>, <Element a at 0x255faccdd68>, <Element a at 0x255faccddb8>, <Element a at 0x255faccde08>]


In [2]:
for link in links:
  newlink = link.get('href')
  if 'http' not in newlink:
    newlink = url + '/' + newlink
  print(newlink)

https://mie.engineering.uiowa.edu/
https://www.uiowa.edu
http://www.stephenbaek.com/idealab
https://www.ccad.uiowa.edu/vsr/
https://www.ccad.uiowa.edu/
https://pda.uiowa.edu/
http://user.engineering.uiowa.edu/~sbaek/about.html/cv.pdf
http://user.engineering.uiowa.edu/~sbaek/about.html/publication.html
https://scholar.google.com/citations?user=dr2krBsAAAAJ&hl=en
http://user.engineering.uiowa.edu/~sbaek/about.html/cv.pdf


## Scraping Flight Status of the CID Airport using BeautifulSoup

In [13]:
import requests
# page = requests.get("https://flycid.com/flight-status/")
page = requests.get("https://webservice.prodigiq.com/wfids/CID/small?rows=18")
page

<Response [200]>

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [15]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->
<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->
<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->
<!--[if IE 9]><html class="lt-ie10"  lang="en" dir="ltr"><![endif]-->
<!--[if (gt IE 9)|(gt IEMobile 7)]><!-->
<html dir="ltr" lang="en" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
 <!--<![endif]-->
 <head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta charset="utf-8"/>
  <title>
   CID Mobile Web
  </title>
 

In [17]:
table = soup.find('table', {'class': 'views-table cols-5'})
print(table)

<table class="views-table cols-5">
<thead>
<tr>
<th class="views-field views-field-title">Flight</th>
<th class="views-field views-field-field-destination">City</th>
<th class="views-field views-field-field-scheduled-time">Time</th>
<th class="views-field views-field-field-revised-time">Claim</th>
<th class="views-field views-field-php">Status</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>AA 9805</td>
<td>Chicago - ORD</td>
<td>8:00 am</td>
<td>2</td>
<td class="ontime">Arrived</td>
</tr>
<tr class="even">
<td>DL 3927</td>
<td>Detroit</td>
<td>9:00 am</td>
<td>1</td>
<td class="ontime">Arrived</td>
</tr>
<tr class="odd">
<td>UA 4601</td>
<td>Chicago - ORD</td>
<td>9:01 am</td>
<td>2</td>
<td class="ontime">Arrived</td>
</tr>
<tr class="even">
<td>DL 3734</td>
<td>Minneapolis</td>
<td>10:11 am</td>
<td>1</td>
<td class="ontime">On time</td>
</tr>
<tr class="odd">
<td>DL 5373</td>
<td>Atlanta</td>
<td>10:34 am</td>
<td>1</td>
<td class="ontime">On time</td>
</tr>
<tr class="even">
<td

In [20]:
tbody = table.find('tbody')

In [23]:
trows = tbody.find_all('tr')

In [32]:
import pandas as pd
df = pd.DataFrame(index=range(len(trows)), columns=['Flight', 'Departure City', 'Arrival Time', 'Baggage Claim', 'Status'])
for i, trow in enumerate(trows):
    titems = trow.find_all('td')    
    df['Flight'][i] = titems[0].contents[0]
    df['Departure City'][i] = titems[1].contents[0]
    df['Arrival Time'][i] = titems[2].contents[0]
    df['Baggage Claim'][i] = titems[3].contents[0]
    df['Status'][i] = titems[4].contents[0]
    
print(df)

     Flight Departure City Arrival Time Baggage Claim       Status
0   AA 9805  Chicago - ORD      8:00 am             2      Arrived
1   DL 3927        Detroit      9:00 am             1      Arrived
2   UA 4601  Chicago - ORD      9:01 am             2      Arrived
3   DL 3734    Minneapolis     10:11 am             1      On time
4   DL 5373        Atlanta     10:34 am             1      On time
5   AA 3473  Chicago - ORD     11:04 am             2  At 11:13 am
6   UA 3794  Chicago - ORD     11:23 am             2      On time
7   AA 4035   Dallas - DFW     12:24 pm             2      On time
8   DL 3878        Detroit     12:49 pm             1      On time
9   AA 5073      Charlotte      1:13 pm             2      On time
10    G4 18      Las Vegas      1:45 pm             1      On time
11   UA 774         Denver      2:03 pm             2      On time
12  DL 4135    Minneapolis      2:04 pm             1      On time
13  AA 3584  Chicago - ORD      2:17 pm             2      On 

### Assignment: Bongo Bus Arrival at the Downtown Interchange


## Get Live Stock Price using `yahoo_fin` API

In [1]:
!pip install requests_html
!pip install --upgrade yahoo_fin

Requirement already up-to-date: yahoo_fin in c:\users\sbaek\appdata\local\continuum\anaconda3\envs\bigdata\lib\site-packages (0.8.2)


In [2]:
from yahoo_fin import stock_info as si

In [3]:
data = si.get_data('NFLX', start_date='01/01/2015', end_date='12/31/2018')

print(data)

                  open        high         low       close    adjclose  \
date                                                                     
2015-01-02   49.151428   50.331429   48.731430   49.848572   49.848572   
2015-01-05   49.258572   49.258572   47.147144   47.311428   47.311428   
2015-01-06   47.347141   47.639999   45.661430   46.501427   46.501427   
2015-01-07   47.347141   47.421429   46.271427   46.742859   46.742859   
2015-01-08   47.119999   47.835712   46.478573   47.779999   47.779999   
...                ...         ...         ...         ...         ...   
2018-12-21  263.829987  264.500000  241.289993  246.389999  246.389999   
2018-12-24  242.000000  250.649994  233.679993  233.880005  233.880005   
2018-12-26  233.919998  254.500000  231.229996  253.669998  253.669998   
2018-12-27  250.110001  255.589996  240.100006  255.570007  255.570007   
2018-12-28  257.940002  261.910004  249.800003  256.079987  256.079987   

                volume ticker  
date 

In [4]:
data.index # gives time stamps
data['volume'].values  # gives values of the column named 'volume'
data[['open','close']].values # gives multiple columns

array([[ 49.15142822,  49.84857178],
       [ 49.25857162,  47.31142807],
       [ 47.34714127,  46.5014267 ],
       ...,
       [233.91999817, 253.66999817],
       [250.11000061, 255.57000732],
       [257.94000244, 256.07998657]])

In [5]:
import matplotlib.pyplot as plt

data = si.get_data('NFLX', start_date='01/01/2015', end_date='12/31/2018')
plt.plot(data.index, data[['open','close']].values) # open, close, high, low, adjclose, volume
plt.title('Netflix Stock Price')
plt.show()

#'open', 'close', 'high', 'low', 'adjclose', 'volume'

C:\Users\sbaek\AppData\Roaming\Python\Python36\site-packages\pandas\plotting\_matplotlib\converter.py:102: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


IndexError: tuple index out of range

< [How to Read and Represent Data](../ica02/How_to_Read_and_Represent_Data.ipynb) | Contents (TODO) | Next Material (TODO) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica03/Data_Mining.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>